In [3]:
import sys
import pathlib
import os
import time
import numpy as np
import matplotlib.pyplot as plt
path_root = pathlib.Path(os.getcwd()).parents[2]
path_module = pathlib.Path(path_root) / 'chiral-client-python'
sys.path.append(str(path_module))
import chiral_client
print(chiral_client.version)

# Username and API Key
user_email = 'new_user_2@gmail.com'
user_token_api = "i7oqcut9lw828uq6xv1cb1kqupllol0u"

# Cloud Configuration
chiral_computing_host = 'localhost'
chiral_computing_port = '20001'
chiral_file_host = '127.0.0.1' 
chiral_file_port = 2121

# create the client
simulation_id = 'lysozyme'
c = chiral_client.client.Client(
    user_email, user_token_api,
    chiral_computing_host, chiral_computing_port,
    chiral_file_host, chiral_file_port
)
job_mgr = chiral_client.GromacsJobManager(simulation_id, './files', c)

# remove the existing directory if it exists
job_mgr.clear_files()
# remote_parent_dir = 'gromacs'
# remote_dir = f"{remote_parent_dir}/{simulation_id}"
# if c.is_remote_dir(remote_parent_dir, simulation_id):
#     c.remove_remote_dir(remote_parent_dir, simulation_id)
# c.create_remote_dir(remote_dir)


# # from pdb to topology
# c.upload_files([('1AKI_clean.pdb', ".", remote_dir)])
# job_id = job_mgr.submit_job(c,
#     'pdb2gmx', "-f 1AKI_clean.pdb -o 1AKI_processed.gro -water spce", "15 0",
#     ["1AKI_clean.pdb"], ["1AKI_processed.gro", "topol.top", "posre.itp"]
# )
# print(f'Job {job_id} created')
# job_mgr.wait_until_completion(c, job_id)
# (output, error) = job_mgr.get_output(c, job_id)
# print(output)
# print(error)


ModuleNotFoundError: No module named 'numpy'